In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FinalProject").getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
from pyspark.sql.types import (IntegerType, DoubleType, StringType)
from pyspark.sql.functions import col,udf,lit,expr,collect_set
import pyspark.sql.functions as F
from pyspark.sql.functions import (avg, col, countDistinct, split, isnan, when, count, udf, round as sqlround)
import math

df = spark.read.format("csv").option("header","true").load("C:/Users/dell/OneDrive - Trường ĐH CNTT - University of Information Technology/Máy tính/BigData/FinalProject/FinalProject_SourceCode/Chennai_Housing_Sale.csv")

def VisualizeCorrelation(dataframe, columns):
  corr_matrix = []
  x_label = []
  y_label = []

  for col in columns:
    ls = []
    for col2 in columns:
      ls.append(df.stat.corr(col, col2))
    corr_matrix.append(ls)
    x_label.append(col)
    y_label.append(col)


  sns.set()
  fig, ax = plt.subplots(figsize = (15,6))
  sns.heatmap(corr_matrix,  xticklabels=x_label, yticklabels=y_label ,annot = True)

#Label Encoding
def label_encode(df, input_col, output_col):
    distinct_values = df.select(input_col).distinct().collect()
    mapping = {row[input_col]: index for index, row in enumerate(distinct_values)}
    mapping_broadcast = spark.sparkContext.broadcast(mapping)
    def label_encode_udf(value):
        return mapping_broadcast.value.get(value, None)
    spark.udf.register("label_encode_udf", label_encode_udf, DoubleType())
    df = df.withColumn(output_col, lit(None).cast(DoubleType()))
    for value, index in mapping.items():
        condition = col(input_col) == lit(value)
        df = df.withColumn(output_col, when(condition, lit(index).cast(DoubleType())).otherwise(col(output_col)))
    df = df.drop(input_col).withColumnRenamed(output_col, input_col)
    return df

#Conver string to double
def convert_string_to_double(df, column_name):
    df = df.withColumn(column_name, col(column_name).cast(DoubleType()))
    return df

#Conver string to int
def convert_string_to_int(df, column_name):
    df = df.withColumn(column_name, col(column_name).cast(IntegerType()))
    return df

#Extract Year from Date
def extract_year_from_date(df, date_column, output_column):
    df = df.withColumn(output_column, expr(f"substring({date_column}, -4)"))
    return df

def math_cal(lines,index,i):
    return (index, double(lines[i]) * double(lines[index]))

def linear_regression(data_train, X_train, y_train):
    value_X = []
    value_y = []
    
    data_train.createOrReplaceTempView("data")
        
    columns_data =""
        
    for i in range(len(X_train)):
        columns_data = columns_data + X_train[i] + ","
    columns_data = columns_data + y_train[0]
        
    sql_data_train = spark.sql("SELECT "+ columns_data +" FROM data")
        
    data_train_rdd = sql_data_train.rdd
        
    n = data_train_rdd.count()
        
    value_X.append(float(n))
        
    map_X_sum = data_train_rdd.map(lambda lines: [(str(index), float(lines[index])) for index in range(len(lines))])
    flatmap_X_sum = map_X_sum.flatMap(lambda lines: lines)
    reduce_X_sum = flatmap_X_sum.reduceByKey(lambda a,b: a+b)
    data_X_sum=reduce_X_sum.collect()
        
        
    for i in range(len(data_X_sum)):
        if(i == len(data_X_sum)-1):
            value_y.append(data_X_sum[i][1])
        else:
            value_X.append(data_X_sum[i][1])
                
    for i in range (len(data_X_sum)-1):
        map_X_vol = data_train_rdd.map(lambda lines:[math_cal(lines, index, i) for index in range(len(lines))])
        flatmap_X_vol = map_X_vol.flatMap(lambda lines: lines)
        reduce_X_vol = flatmap_X_vol.reduceByKey(lambda a,b: a+b)
        data_X_vol = reduce_X_vol.collect()
        
        value_X.append(data_X_sum[i][1])
            
        for j in range(len(data_X_vol)):
            if(j == len(data_X_vol) - 1):
                value_y.append(data_X_vol[j][1])
            else:
                value_X.append(data_X_vol[j][1])
                    
    vector_X = np.asarray(value_X, dtype=np.float32)
    vector_X_convert=np.reshape(vector_X, (len(X_train)+1, len(X_train)+1))

    vector_y = np.asarray(value_y, dtype=np.float32)
    vector_y_convert=np.reshape(vector_y, (len(X_train)+1, 1))

    coefficient = np.linalg.lstsq(vector_X_convert, vector_y_convert)[0]
        
    return coefficient

def predict(data_test, coefficient):
    y=coefficient[0].item()
    for i in range(len(data_test.columns)-1):
        y = y + data_test[i+1]*coefficient[i+1].item()
    return y

def R_square(df_predict):
    avg_price = df_predict.select('price').summary('mean').collect()[0][1]
    df_predict_avg= df_predict.withColumn("price_avg_log",(col("price")-avg_price)*(col("price")-avg_price))
    avg_price_avg_log=df_predict_avg.select('price_avg_log').summary('mean').collect()[0][1]
    sst=float(avg_price_avg_log)*float(df_predict_avg.select('price_avg_log').count())
    
    df_predict_avg_price= df_predict.withColumn("price_predict_avg_log",(col("price")-col("price_predict"))*(col("price")-col("price_predict")))
    avg_price_predict_avg_log = df_predict_avg_price.select('price_predict_avg_log').summary('mean').collect()[0][1]
    sse = float(avg_price_predict_avg_log)*float(df_predict_avg_price.select('price_predict_avg_log').count())
    
    R_square=1-(sse/sst)
    
    return R_square

def RMSD(df_predict, n):
    
    df_predict_avg_price= df_predict.withColumn("price_predict_avg_log",(col("price")-col("price_predict"))*(col("price")-col("price_predict")))
    avg_price_predict_avg_log = df_predict_avg_price.select('price_predict_avg_log').summary('mean').collect()[0][1]
    sse = float(avg_price_predict_avg_log)*float(df_predict_avg_price.select('price_predict_avg_log').count())
    
    return math.sqrt(sse/n)

In [ ]:
df.printSchema()

In [ ]:
# Check for null values in each column
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show(truncate=False)

In [ ]:
df_distinct_counts = df.select([countDistinct(col(c)).alias(c) for c in df.columns])

df_distinct_counts.show()

In [ ]:
#Delete column PRT_ID
df = df.drop("PRT_ID")

In [ ]:
df = extract_year_from_date(df, date_column="DATE_SALE", output_column="YEAR_SALE")
df = df.drop("DATE_SALE")

df = extract_year_from_date(df, date_column="DATE_BUILD", output_column="YEAR_BUILD")
df = df.drop("DATE_BUILD")
df.show(10)

In [ ]:
columns = ["QS_ROOMS", "QS_BATHROOM", "QS_BEDROOM", "QS_OVERALL"]

for col_name in columns:
    df = convert_string_to_double(df, col_name)


In [ ]:
columns = ["INT_SQFT", "DIST_MAINROAD", "N_BEDROOM", "N_BATHROOM",
                   "N_ROOM", "REG_FEE", "COMMIS", "SALES_PRICE", "YEAR_SALE", "YEAR_BUILD"]

for col_name in columns:
    df = convert_string_to_int(df, col_name)


In [ ]:
columns = ["AREA", "BUILDTYPE", "UTILITY_AVAIL", "STREET", "PARK_FACIL", "MZZONE", "SALE_COND"]

for col_name in columns:
    df = label_encode(df, input_col=col_name, output_col=f"{col_name}_Label")

df.printSchema()

In [ ]:
df.show(10)

In [ ]:
#Tìm hệ số tương quan giữa các cột với cột price
mid = int((len(df.columns) + 1) / 2) 
listCol1 = df.columns[0:mid]
listCol2 = df.columns[mid:len(df.columns)]
listCol2.insert(0, df.columns[0])

VisualizeCorrelation(df, listCol1)
#VisualizeCorrelation(df, listCol2)

In [ ]:
VisualizeCorrelation(df, listCol2)

In [ ]:
df.printSchema()

In [ ]:
df.show(30)

In [ ]:
X_train = df.columns
X_train.remove('price')

y_train = ['price']

In [ ]:
data_train = df.limit(int(df.count() * 0.7))
data_test = df.subtract(data_train)

coefficient = linear_regression(data_train,X_train,y_train)



In [ ]:
df_predict = data_test.withColumn("price_predict",predict(data_test,coefficient))
df_predict.show(30)   

In [ ]:
print("R_square: "+ str(R_square(df_predict)))
print("RMSD: "+str(RMSD(df_predict,data_test.count())))